# Cyberbullying📝

![Banner](./assets/banner.jpeg)

## Topic
*What problem are you (or your stakeholder) trying to address?*
📝 <!-- Answer Below -->
The problem that I intend to address with this research and analysis is the increase in cyberbullying that has proven to be detrimental, especially to the younger demographic. Due to the serious psychological and emotional effects it can have on victims, which can include despair, anxiety, and in severe cases, suicide,this issue is important. It is imperative to combat cyberbullying in order to create a supportive and safe digital environment as technology develops and more people, particularly teenagers, become active online.

## Project Question
*What specific question are you seeking to answer with this project?*
*This is not the same as the questions you ask to limit the scope of the project.*
📝 <!-- Answer Below -->

-What is the relationship between recorded suicide cases and cyberbullying

-Is there a trend by age on the observed sources and targets of cyberbullying online 

-How can the social media platforms limit or get rid of extreme cyberbullying online 

## What would an answer look like?
*What is your hypothesized answer to your question?*
📝 <!-- Answer Below -->

-There is a positive correlation between the increase in suicide rates, and cyberbullying activities online This implies that cyberbullying has has had a significant impact on the rate of suicide over the last 10 years.

## Data Sources
*What 3 data sources have you identified for this project?*
*How are you going to relate these datasets?*
📝 <!-- Answer Below -->

The Data sources I have identified for this project include:

-Kaggle datasets from suicide-watch for suicide trend from 1985-2016
-Kaggle datasets for compiled cyberbullying tweets with word scraping
-Webscrapped suicide demographic and behavioral data from cdc website 


## Approach and Analysis
*What is your approach to answering your project question?*
*How will you use the identified data to answer your project question?*
📝 <!-- Start Discussing the project here; you can add as many code cells as you need -->
I will use the data from Kaggle to identify the target demographic for the cyberbullying tweets, I then will analyse that data against the suicide data from both Kaggle and cdc to determine the average age group where suicide is more imminent and using the age and gender demographics to find a correlation by linear Regression and visualizations.

In [14]:
import pandas as pd
import opendatasets as od
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import joblib
# Python ≥3.10 is required
import sys
assert sys.version_info >= (3, 10)

# Scikit Learn imports
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.style.use("bmh")

# to make this notebook's output stable across runs
np.random.seed(42)



In [1]:
# Start your code here

tweets_url= "https://www.kaggle.com/datasets/soorajtomar/cyberbullying-tweets"

od.download (tweets_url, data_dir= "./data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading cyberbullying-tweets.zip to ./data\cyberbullying-tweets


100%|██████████| 441k/441k [00:00<00:00, 3.85MB/s]

In [3]:
#suicide_url= "https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch"

#od.download (suicide_url, data_dir= "./data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading suicide-watch.zip to ./data\suicide-watch


100%|██████████| 60.6M/60.6M [00:13<00:00, 4.82MB/s]


In [16]:
suicide_df = pd.read_csv ('data/suicide.csv')

suicide_df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [7]:
cdc_url= "https://www.cdc.gov/mmwr/volumes/69/su/su6901a6.htm"
page= requests.get(cdc_url)
soup= BeautifulSoup (page.content, 'html.parser')
tables = soup.find_all('table')

cdc_data_df = pd.read_html (str(tables))

cdc_data_df

C:\Users\zsams\AppData\Local\Temp\ipykernel_22048\3141362488.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  cdc_data_df = pd.read_html (str(tables))


[                                             Behavior  \
 0             Seriously considered attempting suicide   
 1                                                 Yes   
 2                                                  No   
 3                                 Made a suicide plan   
 4                                                 Yes   
 5                                                  No   
 6                                   Attempted suicide   
 7                                                 Yes   
 8                                                  No   
 9   Made a suicide attempt requiring medical treat...   
 10                                                Yes   
 11                                                 No   
 
                                     Female % (95% CI)  \
 0             Seriously considered attempting suicide   
 1                                    24.1 (22.3‒26.0)   
 2                                    75.9 (74.0‒77.7)   
 3          

## Resources and References
*What resources and references have you used for this project?*
📝 <!-- Answer Below -->


-https://www.kaggle.com/datasets/russellyates88/suicide-rates-overview-1985-to-2016/

-https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch

-https://medium.com/linkit-intecs/how-to-upload-large-files-to-github-repository-2b1e03723d2

-https://www.pewresearch.org/search/cyberbullying

# ⚠️ Make sure you run this cell at the end of your notebook before every submission!
!jupyter nbconvert --to python source.ipynb